<a href="https://colab.research.google.com/github/friyk/DSL-30.110/blob/main/T03/T03_CmodA7_UART_ADC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install moviepy
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import requests

  if event.key is 'enter':



# T03 - CMOD A7 UART & ADC

|Group members|Student ID|
|---|---|
|Chua Kevin Subong|1007002|
|Kaung Khant Htet|1003437|
|Ng Au Hern Wesley|1006888|
|Pan Ziyue|1006658|
|Shum Hei Lam|1009861|

## Objective  
Develop a Verilog-based ADC driver for the **CMOD A7 FPGA board** to interface with an SPI-based **MCP3202 ADC** (on a PMOD extension board). The project includes:  
- Displaying ADC results on a 4-digit 7-segment display.  
- Transmitting data to a PC via UART.  

---

## Key Features  
1. **Reset Functionality**  
   - Press **BTN1** to reset the system and reinitialize components.  

2. **SPI ADC Driver**  
   - Interface with MCP3202 ADC via SPI protocol.  
   - Ensure accurate timing and data handling for sampling.  

3. **ADC Value Display**  
   - Process sampled data and display results on the 7-segment display.  

4. **UART Transmission**  
   - Transmit ADC data to a PC with configurable baud rate (e.g., 115200).  

---

## Deliverables  
- **Verilog Code** for:  
  - SPI ADC driver  
  - 7-segment display controller  
  - UART transmitter  
- **Demonstration Video** (MP4, ≤1 min) showing:  
  - Light sensor sampling → 7-segment display  
  - Light sensor sampling → UART transmission to PC  
  - Customized feature (optional)  

---

## Instructions  

### Hardware Preparation  
1. **7-Segment Display**  
   - Use the **3642AS-1** 4-digit display.

| FPGA Pin  | Segment/Digit Pin | Segment Label    |
|-----------|-------------------|------------------|
| PIO48     | 7                 | Segment-B        |
| PIO47     | 8                 | Dig3 (Digit 3)   |
| PIO46     | 9                 | Dig2 (Digit 2)   |
| PIO45     | 10                | Segment-F        |
| PIO44     | 11                | Segment-A        |
| PIO43     | 12                | Dig1 (Digit 1)   |
| PIO42     | 1                 | Segment-E        |
| PIO41     | 2                 | Segment-D        |
| PIO40     | 3                 | Segment-DP       |
| PIO39     | 4                 | Segment-C        |
| PIO38     | 5                 | Segment-G        |
| PIO37     | 6                 | Dig4 (Digit 4)   |

   - Connection Demo
   ![Connect Schematic](https://github.com/pe8sutd/Colab_DSL_Practice25/blob/main/figure/T03_Connect.JPEG?raw=1)

2. **MCP3202 ADC Extension Board**  
   - Channel selection via SW1:  
     - **Left**: External signals (EX0/EX1 → CH0/CH1)  
     - **Right**: Onboard NTC (temp sensor) and LIGHT (ambient light sensor) → CH1/CH0.
     - **[Schematic](https://github.com/pe8sutd/Colab_DSL_Practice25/blob/0865141e546e601a029f7c385b6bd2fd61b6fdb0/Demo_Code/T03_CmodA7_Uart_ADC/Figure/Schematic_M2_Temp_Light_Sensor.pdf)**: https://github.com/pe8sutd/Colab_DSL_Practice25/blob/0865141e546e601a029f7c385b6bd2fd61b6fdb0/Demo_Code/T03_CmodA7_Uart_ADC/Figure/Schematic_M2_Temp_Light_Sensor.pdf

     ![Ext Board Schematic](https://github.com/pe8sutd/Colab_DSL_Practice25/blob/main/figure/T03_Ext.png?raw=1)

### FPGA Project  
1. **Vivado Setup**  
   - Create a project for **CMOD A7-35T**.  

2. **Verilog Implementation**  

3. **Constraint File (.xdc)**  

4. **Bitstream & Programming**  
   - Generate and flash the bitstream to the FPGA.  

5. **Testing & Documentation**  
   - Debug using onboard LEDs/UART output.  

---

## Simulation on Colab
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pe8sutd/Colab_DSL_Practice25/blob/main/Simulation/simT03_CmodA7_UART_ADC.ipynb)

---

## Deadline  
- Submit via edimension by the specified due date.  

---

## Grading  
- **Functionality** (50%): SPI/UART accuracy, display working.  
- **Code Clarity** (30%): Clean Verilog, proper constraints.  
- **Documentation** (20%): Report, diagrams, challenges.  
- **Bonus**: Innovativeness (up to +30%).  

- The report should present how AI is used in the design (if applicable). The guideline: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/pe8sutd/Colab_DSL_Practice25/blob/main/G01_Used_of_AI_in_Design.ipynb)


---




# Submission


## Wiring

![Wiring Diagram](https://github.com/friyk/DSL-30.110/blob/main/T03/photo_2025-04-03_12-37-07.jpg?raw=true)

  ## **Verilog Code** for:  




  ### 7 segment display controller

In [ ]:
%verilog
// `timescale 1ns / 1ps
//////////////////////////////////////////////////////////////////////////////////
// Engineer: MaoYang
// Create Date: 18.03.2024 11:57:44
// Module Name: drv_segment
//////////////////////////////////////////////////////////////////////////////////

module drv_segment(
    input rstn,
    input clk500hz,
    input [15:0] bcd_num,
    output [3:0] an,
    output [7:0] segment
);

reg [7:0] segment_r;
reg [3:0] an_r;
assign segment = ~ segment_r;
reg [3:0] cur_num_r;        //Register - BCD Number Display at this moment;
assign an = ~an_r;
//Drive 7Segment Anode;
//When an_r == 0001, DIG4 will turn on;
//When an_r == 0001, at posedge clk500hz, an_r will be set to 0010(DIG3 ON);
//When an_r == 0010, at posedge clk500hz, an_r will be set to 0100(DIG2 ON);
//....
//DIG4 -> DIG3 -> DIG2 -> DIG1 -> DIG4 -> DIG3 -> DIG2 -> ...;
always @(negedge rstn,posedge clk500hz)begin
    if(!rstn)begin
        an_r <= 4'b0000;    //When system reset, empty all display;
    end
    else begin
        case(an_r)
        4'b0001: an_r <= 4'b0010;   //DISPLAY ON DIG3
        4'b0010: an_r <= 4'b0100;   //DISPLAY ON DIG2
        4'b0100: an_r <= 4'b1000;   //DISPLAY ON DIG1
        default: an_r <= 4'b0001;   //DISPLAY ON DIG4
        endcase
    end
end

//When DIG4 on, BCD Number Display at this moment is bcd_num[3:0];  (i.e Stop Watch - Second Unit)
//When DIG3 on, BCD Number Display at this moment is bcd_num[7:4];  (i.e Stop Watch - Second Decade)
//When DIG2 on, BCD Number Display at this moment is bcd_num[11:8]; (i.e Stop Watch - Minute Unit)
//When DIG1 on, BCD Number Display at this moment is bcd_num[15:12];(i.e Stop Watch - Minute Decade)
always @(an_r,bcd_num)begin
    case(an_r)
        4'b0001: cur_num_r <= bcd_num[3:0];
        4'b0010: cur_num_r <= bcd_num[7:4];
        4'b0100: cur_num_r <= bcd_num[11:8];
        4'b1000: cur_num_r <= bcd_num[15:12];
        default: cur_num_r <= 4'b0;
    endcase
end

//Decode BCD NUM into corrosponding 7Segment Code;
always @(cur_num_r) begin
    case(cur_num_r)
        4'h0:segment_r <= 8'hc0;    //NUM "0"
        4'h1:segment_r <= 8'hf9;    //NUM "1"
        4'h2:segment_r <= 8'ha4;    //NUM "2"
        4'h3:segment_r <= 8'hb0;    //NUM "3"
        4'h4:segment_r <= 8'h99;    //NUM "4"
        4'h5:segment_r <= 8'h92;    //NUM "5"
        4'h6:segment_r <= 8'h82;    //NUM "6"
        4'h7:segment_r <= 8'hF8;    //NUM "7"
        4'h8:segment_r <= 8'h80;    //NUM "8"
        4'h9:segment_r <= 8'h90;    //NUM "9"
        4'ha:segment_r <= 8'h88;    //NUM "A" - HEX NUMBER DISPLAY
        4'hb:segment_r <= 8'h83;    //NUM "b"
        4'hc:segment_r <= 8'hc6;    //NUM "C"
        4'hd:segment_r <= 8'ha1;    //NUM "D"
        4'he:segment_r <= 8'h86;    //NUM "E"
        4'hf:segment_r <= 8'h8e;    //NUM "F"
        default: segment_r <= 8'hff;
    endcase
end

endmodule


### SPI ADC driver

In [ ]:
%verilog
/*
 * Module: drv_mcp3202
 * Date : 2024/03/21
 * Author : Maoyang
 * Description:
 * This Verilog module implements an SPI interface for the MCP3202 Analog-to-Digital Converter (ADC).
 * It manages the communication process through a finite state machine (FSM) to read analog data and convert it to a digital format.
 *
 * Inputs:
 * - rstn: Active low reset signal.
 * - clk: Clock signal.
 * - ap_ready: Signal indicating the application is ready for data processing.
 * - mode: 2-bit input to select the ADC channel and configuration.
 *          localparam  SINGLE_CHAN0   = 2'b10; - CHANNEL 0;
 *          localparam  SINGLE_CHAN1   = 2'b11; - CHANNEL 1;
 *          localparam  DIFFER_CHAN01  = 2'b00;  - DIFFERENTIAL CHANNEL 01
 *          localparam  DIFFER_CHAN10  = 2'b01;  - DIFFERENTIAL CHANNEL 10
 * - port_din: Serial data input from the ADC.
 *
 * Outputs:
 * - ap_vaild: Signal that indicates valid data is available.
 * - data: 12-bit output holding the ADC conversion result.
 * - port_dout: Serial data output to the ADC.
 * - port_clk: Clock signal for the ADC.
 * - port_cs: Chip select signal for the ADC, active low.
 *
 * Functionality:
 * - The module configures the ADC based on the mode input.
 * - It uses an FSM to control the SPI communication process, including sending control bits, reading the ADC data, and signaling when new data is available.
 * - The ADC conversion result is made available as a 12-bit output.
 * - The module ensures synchronization with the external ADC device through careful management of clock and control signals.
 *
 * Implementation Details:
 * - The Data_Transmit wire is used to send control signals to the ADC.
 * - The Data_Receive register captures the ADC output.
 * - The FSM transitions through several states (IDLE, WRITE, READ, STOP) to manage the entire data transfer process.
 * - The cnter_writ and cnter_read registers are used to track progress through the transmit and receive phases, respectively.
 */

module drv_mcp3202(
    input rstn,
    input clk,
    input   ap_ready,
    output  reg ap_vaild,
    input   [1:0] mode,
    output  [11:0] data,

    input   port_din,
    output  reg port_dout,
    output  port_clk,
    output  reg port_cs
);

wire    [3:0]      Data_Transmit; // 4 bits CONTROL;
reg     [12:0]     Data_Receive;  // 1 bit NULL + 12 bits DATA;

assign Data_Transmit[3]    = 1'b1;
assign Data_Transmit[0]    = 1'b1;
assign Data_Transmit[2:1] = mode;

reg [1:0]   fsm_statu,fsm_next;
localparam FSM_IDLE = 2'b00;
localparam FSM_WRIT = 2'b10;
localparam FSM_READ = 2'b11;
localparam FSM_STOP = 2'b01;

reg [1:0] cnter_writ;
reg [3:0] cnter_read;

//FSM statu transfer;
always @(posedge clk, negedge rstn) begin
    if (!rstn)  fsm_statu <= FSM_IDLE;
    else        fsm_statu <= fsm_next;
end

//FSM Transfer Condition;
always @(*)begin
    if(!rstn) fsm_next <= FSM_IDLE;
    else begin
        case (fsm_statu)
            FSM_IDLE : fsm_next <= (ap_ready)? FSM_WRIT : FSM_IDLE;
            FSM_WRIT : fsm_next <= (2'd0 == cnter_writ)? FSM_READ : FSM_WRIT;
            FSM_READ : fsm_next <= (2'd0 == cnter_read)? FSM_STOP : FSM_READ;
            FSM_STOP : fsm_next <= (!ap_ready)? FSM_STOP : FSM_IDLE;
            default  : fsm_next <= FSM_IDLE;
        endcase
    end
end

//FSM Output - SPI Write Data
always @(negedge rstn,negedge clk)begin
    if (!rstn) begin
        cnter_writ  <= 2'd3;
        port_dout   <= 1'b1;
        port_cs     <= 1'b1;
    end else begin
        case (fsm_statu)
            FSM_IDLE : begin
                cnter_writ  <= 2'd3;
                port_dout   <= 1'b1;
                port_cs     <= 1'b1;
            end
            FSM_WRIT : begin
                port_cs     <= 1'b0;
                port_dout   <= Data_Transmit[cnter_writ];
                cnter_writ  <= cnter_writ - 1'b1;
            end
            FSM_READ : begin
                port_cs     <= 1'b0;
                port_dout   <= 1'b1;
            end
            FSM_STOP : port_cs     <= 1'b1;
            default  : ;
        endcase
    end
end

//FSM Output - SPI Read  Data
always @(negedge rstn,posedge clk)begin
    if (!rstn) begin
        cnter_read  <= 4'd13;
        Data_Receive <= 13'h00;
        ap_vaild = 1'b0;
    end else begin
        case (fsm_statu)
            FSM_IDLE : begin
                ap_vaild = 1'b0;
                cnter_read  <= 4'd13;
            end
            FSM_WRIT : begin
                Data_Receive <= 13'h00;
            end
            FSM_READ : begin
                cnter_read <= cnter_read - 1'b1;
                Data_Receive[cnter_read] <= port_din;
            end
            FSM_STOP : ap_vaild = 1'b1;
            default  : ;
        endcase
    end
end

assign port_clk = clk | port_cs;
assign data = Data_Receive[11:0];

endmodule

### UART Transmitter

In [ ]:
%verilog
// Module Name: uart_tx
// Date : 2024/03/24
// Version : V0.1
// Author : Maoyang
// Description: Implements a UART transmitter with optional parity bit functionality. This module encodes and transmits
// data over a serial line using the UART protocol. It supports transmitting data with start, stop, and optional parity bits,
// and signals the completion of transmission.

// Inputs:
// clk: System clock signal for synchronizing the transmission process (Baud Rate).
// ap_rstn: Asynchronous, active low reset signal to initialize or reset the module's internal states.
// ap_ready: Input signal indicating readiness to start data transmission. When high, transmission begins.
// pairty: Input signal to enable (when high) or disable (when low) parity bit generation and transmission.
// data: 8-bit data input to be transmitted over UART.

// Outputs:
// ap_vaild: Output signal that indicates the completion of a data transmission cycle.
// tx: Serial output transmitting the encoded data along with start, stop, and optional parity bits.

// Local Parameters:
// FSM_IDLE, FSM_STAR, FSM_TRSF, FSM_PARI, FSM_STOP: Represent the states of the finite state machine (FSM) controlling
// the UART transmission process, from idle, through start bit, data transmission, optional parity bit, and stop bit.

// Internal Registers:
// fsm_statu: Holds the current state of the FSM.
// fsm_next: Determines the next state of the FSM based on the current state and input signals.
// cnter: Counter used during the data transmission state to index through the data bits.

// Behavioral Blocks:
// 1. fsm statu transfer: Sequential logic block that updates the current state of the FSM on each positive clock edge or
//    on negative edge of ap_rstn. Resets to FSM_IDLE on reset.
// 2. fsm conditional transfer: Combinatorial logic block that determines the next state of the FSM based on current
//    conditions like ap_ready signal, counter value, and parity configuration.
// 3. fsm - output: Sequential logic block that performs actions based on the current FSM state, including setting the
//    tx output according to the data bits, generating a parity bit if enabled, and indicating the end of transmission
//    through ap_vaild signal. Also handles the initialization of internal signals on reset.

// Note: This module is designed to be synthesized and integrated into larger systems requiring UART transmission
// capabilities, with configurable support for parity bit for error detection.
module drv_uart_tx(
    input   clk,
    input   ap_rstn,
    input   ap_ready,
    output  reg ap_vaild,
    output  reg tx,
    input   pairty,
    input  [7:0] data
);

localparam  FSM_IDLE = 3'b000,
            FSM_STAR = 3'b001,
            FSM_TRSF = 3'b010,
            FSM_PARI = 3'b011,
            FSM_STOP = 3'b100;

reg [2:0] fsm_statu;
reg [2:0] fsm_next;
reg [2:0] cnter;

//fsm statu transfer;
always @(posedge clk, negedge ap_rstn) begin
    if (!ap_rstn)begin
        fsm_statu <= FSM_IDLE;
    end else begin
        fsm_statu <= fsm_next;
    end
end

//fsm conditional transfer;
always @(*)begin
    if(!ap_rstn)begin
        fsm_next <= FSM_IDLE;
    end else begin
        case(fsm_statu)
            FSM_IDLE:begin
                fsm_next <= (ap_ready) ? FSM_STAR : FSM_IDLE;
            end
            FSM_STAR: fsm_next <= FSM_TRSF;
            FSM_TRSF:begin
                fsm_next <= (cnter == 3'h7) ? (pairty?FSM_PARI:FSM_STOP) : FSM_TRSF;
            end
            FSM_PARI: fsm_next <= FSM_STOP;
            FSM_STOP:begin
                fsm_next <= (!ap_ready) ? FSM_IDLE : FSM_STOP;
            end
            default: fsm_next <= FSM_IDLE;
        endcase
    end
end

//fsm - output
always @(posedge clk, negedge ap_rstn)begin
    if(!ap_rstn)begin
        ap_vaild <= 1'b0;
        tx <= 1'b1;
        cnter <= 3'h0;
    end else begin
        case (fsm_statu)
            FSM_IDLE: begin
                tx <= 1'b1;
                ap_vaild <= 1'b0;
            end
            FSM_STAR: begin
                tx <= 1'b0;
                cnter <= 3'h0;
            end
            FSM_TRSF: begin
                tx <= data[cnter];
                cnter <= cnter + 1'b1;
            end
            FSM_PARI: tx <= (^data); //Parity Check - ODD Check;
            FSM_STOP: begin
                tx <= 1'b1;         //Stop Bit;
                ap_vaild <= 1'b1;
            end
        endcase
    end
end

endmodule

  ## **Demonstration Video**

  - Light sensor sampling → 7-segment display  
  - Light sensor sampling → UART transmission to PC  
  - Customized feature (optional)  

### Reset functionality

In [2]:
video_url = "https://github.com/friyk/DSL-30.110/blob/main/T03/Reset%20Button.mp4?raw=true"
video_path = "Reset_Button.mp4"  # Local file name

response = requests.get(video_url, stream=True)
with open(video_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            file.write(chunk)

video = VideoFileClip(video_path)  # Use the local file name
video_html = video.ipython_display(width=640, autoplay=False, loop=False)  # Customize as needed
HTML(video_html)

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


TypeError: 'HTML2' object is not subscriptable

### Light sensor sampling → 7-segment display

#Appendix


### 7-Segment Display Schematic
![7-Segment Display Schematic](https://github.com/pe8sutd/Colab_DSL_Practice25/blob/main/figure/T03_7Segment.png?raw=1)


### MCP3202 ADC  
- **Datasheet**: [MCP3202 Datasheet](https://pdf1.alldatasheet.com/datasheet-pdf/view/74935/MICROCHIP/MCP3202.html)  
- **PCB Layout**:  
  - Use headers J1/J2 for breadboard mounting.  
  - Input options: SMA port or 2.54mm header.  

### Demo Figures  
1. **ADC Sampling & UART Transmission**

   ![ADC-UART](https://github.com/pe8sutd/Colab_DSL_Practice25/blob/main/figure/T03_ADC_UART.png?raw=1)  

2. **Light Sensor Response**  

   ![Light Sensor](https://github.com/pe8sutd/Colab_DSL_Practice25/blob/main/figure/T03_Light.png?raw=1)  

---

## References  
- **Demo Code**: [CMODA7_Demo_MCP3202_SPI_Verilog](./Demo_Code/T03_CmodA7_Uart_ADC)  
- **Vivado Guide**: [Vivado Guide Link](https://classroom.google.com/u/0/c/NzM2NTAyNzI3NDY5/m/NzI1OTgzMjk4NTI2/details)  
- **EasyEDA Tutorial**: PCB design guidelines for CMOD A7.  